In [ ]:
# 속도 향상 삭제 금지

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
)
import time
import json
import traceback

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.5  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")

def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임이 로드될 때까지 기다림
            try:
                iframe_seat = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("Switched to iframe 'ifrmSeatFrame'")
            except TimeoutException:
                print("iframe 'ifrmSeatFrame'을 찾지 못했습니다. 다시 시도합니다.")
                time.sleep(monitoring_interval)
                attempts += 1
                continue  # 다음 시도로 넘어감

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 바로 버튼 클릭 시도
                try:
                    confirm_button = driver.find_element(By.XPATH, "//a[@href='javascript:ChoiceEnd();']")
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# '다음단계' 버튼 클릭 함수 정의
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False
    return True

# '다음단계' 버튼 클릭
if click_next_step():
    print("다음 단계로 이동했습니다.")
else:
    print("다음 단계로 이동하지 못했습니다.")

# 추가 작업이 필요하다면 여기에 작성합니다.

In [ ]:
# 글로벌 도전 성공, 삭제 금지

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import json
import traceback
import datetime
import time
import requests

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
MAX_ATTEMPTS = 100000
MONITORING_INTERVAL = 0.5  # 초
WAIT_TIME = 10  # 기본 대기 시간 (초)

# WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, WAIT_TIME)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('http://ticket.yes24.com/Pages/English/Perf/FnPerfDeail.aspx?IdPerf=51057')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r', encoding='utf-8') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            # 'sameSite' 속성 제거 (필요 시)
            cookie.pop('sameSite', None)
            # 쿠키에 필요한 도메인 정보가 있는지 확인
            if 'domain' not in cookie:
                cookie['domain'] = 'ticket.yes24.com'
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 페이지가 로드되고 특정 요소가 나타날 때까지 대기
try:
    # 예: 'hlkPurchase' 버튼이 클릭 가능할 때까지 대기
    wait.until(EC.element_to_be_clickable((By.ID, 'hlkPurchase')))
    print("페이지 로드 완료")
except TimeoutException:
    print("페이지 로드 대기 시간 초과")
    driver.quit()
    exit()

# Booking 버튼 클릭
try:
    booking_button = driver.find_element(By.ID, 'hlkPurchase')
    booking_button.click()
    print("Booking 버튼 클릭 완료")
except NoSuchElementException:
    print("Booking 버튼을 찾을 수 없습니다.")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 새 창으로 제어 전환
try:
    wait.until(EC.number_of_windows_to_be(2))  # 새 창이 열릴 때까지 대기
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except TimeoutException:
    print("새 창이 열리지 않았습니다.")
    driver.quit()
    exit()

# 팝업 처리 after clicking second booking
handle_alert()

# 날짜 선택
try:
    # 날짜 버튼이 로드되고 클릭 가능할 때까지 대기
    date_button = wait.until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 실제 날짜 ID로 수정 필요
    )
    date_button.click()
    print("날짜 선택 완료")
except TimeoutException:
    print("날짜 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except TimeoutException:
    print("회차 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = wait.until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except TimeoutException:
    print("좌석 선택 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        wait_short = WebDriverWait(driver, 2)
        wait_short.until(
            lambda d: 'selected' in seat_element.get_attribute('class').lower()
        )
        print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
        return True

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except TimeoutException:
        print("'좌석선택완료' 버튼을 찾거나 클릭할 수 없습니다.")
        traceback.print_exc()
        return False
    except Exception as e:
        print(f"'좌석선택완료' 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False
    
# 텔레그램 메시지 전송 함수
def send_telegram_message(message):
    bot_token = '7616270521:AAGXL0x51EOH6iqbuBN9TfFqOWqoSphjKlc'
    chat_id = '7889718434'
    
    send_message_url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    
    params = {
        'chat_id': chat_id,
        'text': message
    }
    
    response = requests.get(send_message_url, params=params)
    
    if response.status_code == 200:
        print("텔레그램 메시지가 성공적으로 전송되었습니다.")
    else:
        print(f"텔레그램 메시지 전송 실패: {response.status_code}, {response.text}")

# 좌석 선택 모니터링 및 시도 함수 (수정된 부분 포함)
def monitor_and_select_seat():
    attempts = 0

    while attempts < MAX_ATTEMPTS:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {MAX_ATTEMPTS}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")  
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            send_telegram_message(f"성공적으로 좌석을 선택했습니다! 좌석 수: {seat_count}")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(MONITORING_INTERVAL)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드되고 클릭 가능할 때까지 대기
        next_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except TimeoutException:
        print("'다음단계' 버튼을 찾거나 클릭할 수 없습니다.")
        traceback.print_exc()
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"스크립트 실행 중 예기치 않은 오류 발생: {e}")
        traceback.print_exc()
    finally:
        # 모든 작업 후 브라우저를 종료하려면 아래 주석을 해제하세요
        # driver.quit()
        pass


쿠키 로드 성공
페이지 로드 완료
Booking 버튼 클릭 완료
새 창으로 제어를 전환했습니다.
날짜 선택 완료
회차 선택 완료
좌석 선택 버튼 클릭 완료
Attempt 1 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 2 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 3 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 4 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 5 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 6 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 7 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 8 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 9 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 10 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 11 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 12 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 13 out of 100000
조건을 만족하는 좌석은 총 0개입니다.
선택 가능한 좌석을 찾지 못했습니다.
Attempt 14 out of 100000
조건을 만족하는 좌석은 

KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import json
import traceback
import datetime
import time
import requests

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
MAX_ATTEMPTS = 100000
MONITORING_INTERVAL = 0.5  # 초
WAIT_TIME = 10  # 기본 대기 시간 (초)

# WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, WAIT_TIME)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('http://ticket.yes24.com/Pages/English/Perf/FnPerfDeail.aspx?IdPerf=51057')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r', encoding='utf-8') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            # 'sameSite' 속성 제거 (필요 시)
            cookie.pop('sameSite', None)
            # 쿠키에 필요한 도메인 정보가 있는지 확인
            if 'domain' not in cookie:
                cookie['domain'] = 'ticket.yes24.com'
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 페이지가 로드되고 특정 요소가 나타날 때까지 대기
try:
    # 예: 'hlkPurchase' 버튼이 클릭 가능할 때까지 대기
    wait.until(EC.element_to_be_clickable((By.ID, 'hlkPurchase')))
    print("페이지 로드 완료")
except TimeoutException:
    print("페이지 로드 대기 시간 초과")
    driver.quit()
    exit()

# Booking 버튼 클릭
try:
    booking_button = driver.find_element(By.ID, 'hlkPurchase')
    booking_button.click()
    print("Booking 버튼 클릭 완료")
except NoSuchElementException:
    print("Booking 버튼을 찾을 수 없습니다.")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 새 창으로 제어 전환
try:
    wait.until(EC.number_of_windows_to_be(2))  # 새 창이 열릴 때까지 대기
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except TimeoutException:
    print("새 창이 열리지 않았습니다.")
    driver.quit()
    exit()

# 팝업 처리 after clicking second booking
handle_alert()

# 날짜 선택
try:
    # 날짜 버튼이 로드되고 클릭 가능할 때까지 대기
    date_button = wait.until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 실제 날짜 ID로 수정 필요
    )
    date_button.click()
    print("날짜 선택 완료")
except TimeoutException:
    print("날짜 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    print("회차 선택 완료")
except TimeoutException:
    print("회차 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = wait.until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except TimeoutException:
    print("좌석 선택 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        wait_short = WebDriverWait(driver, 2)
        wait_short.until(
            lambda d: 'selected' in seat_element.get_attribute('class').lower()
        )
        print(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
        return True

    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except TimeoutException:
        print("'좌석선택완료' 버튼을 찾거나 클릭할 수 없습니다.")
        traceback.print_exc()
        return False
    except Exception as e:
        print(f"'좌석선택완료' 버튼 클릭 오류: {e}")
        traceback.print_exc()
        return False
    
# 텔레그램 메시지 전송 함수
def send_telegram_message(message):
    bot_token = '7616270521:AAGXL0x51EOH6iqbuBN9TfFqOWqoSphjKlc'
    chat_id = '7889718434'
    
    send_message_url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    
    params = {
        'chat_id': chat_id,
        'text': message
    }
    
    response = requests.get(send_message_url, params=params)
    
    if response.status_code == 200:
        print("텔레그램 메시지가 성공적으로 전송되었습니다.")
    else:
        print(f"텔레그램 메시지 전송 실패: {response.status_code}, {response.text}")

# 좌석 선택 모니터링 및 시도 함수 (수정된 부분 포함)
def monitor_and_select_seat():
    attempts = 0

    while attempts < MAX_ATTEMPTS:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {MAX_ATTEMPTS}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @title]")  
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                # 선택 가능한 좌석을 순차적으로 시도
                for seat in seats:
                    # 좌석 선택 시도
                    success = select_seat(seat)
                    if success:
                        # 좌석선택완료 버튼 클릭 시도
                        confirm_success = click_confirm()
                        if confirm_success:
                            print("좌석 선택 완료 및 확인 버튼 클릭 성공.")
                            send_telegram_message(f"성공적으로 좌석을 선택했습니다! 좌석 수: {seat_count}")
                            return True  # 좌석 선택 및 확인 버튼 클릭 성공
                        else:
                            print("좌석 선택 완료 버튼 클릭 실패.")
                            continue  # 다음 좌석으로 넘어감
                    else:
                        print("좌석 선택 실패. 다시 시도합니다.")
                        continue  # 다음 좌석으로 넘어감

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(MONITORING_INTERVAL)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        print("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드되고 클릭 가능할 때까지 대기
        next_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except TimeoutException:
        print("'다음단계' 버튼을 찾거나 클릭할 수 없습니다.")
        traceback.print_exc()
        return False
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도 및 '다음단계' 버튼 클릭
def main():
    # 좌석 선택 시도
    if monitor_and_select_seat():
        print("좌석 선택에 성공했습니다.")
        # '다음단계' 버튼 클릭
        if click_next_step():
            print("다음 단계로 이동했습니다.")
        else:
            print("다음 단계로 이동하지 못했습니다.")
    else:
        print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

    # 브라우저 유지 또는 종료
    # 필요에 따라 주석을 해제하세요
    # driver.quit()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"스크립트 실행 중 예기치 않은 오류 발생: {e}")
        traceback.print_exc()
    finally:
        # 모든 작업 후 브라우저를 종료하려면 아래 주석을 해제하세요
        # driver.quit()
        pass


In [4]:
# 다른 아티스트 테스트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import json
import traceback
import datetime
import requests

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
MAX_ATTEMPTS = 10000
MONITORING_INTERVAL = 0.5  # 초
WAIT_TIME = 10  # 기본 대기 시간 (초)

# WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, WAIT_TIME)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('http://ticket.yes24.com/Pages/English/Perf/FnPerfDeail.aspx?IdPerf=51131')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r', encoding='utf-8') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            cookie.pop('sameSite', None)
            if 'domain' not in cookie:
                cookie['domain'] = 'ticket.yes24.com'
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 페이지가 로드되고 특정 요소가 나타날 때까지 대기
try:
    wait.until(EC.element_to_be_clickable((By.ID, 'hlkPurchase')))
    print("페이지 로드 완료")
except TimeoutException:
    print("페이지 로드 대기 시간 초과")
    driver.quit()
    exit()

# Booking 버튼 클릭
try:
    booking_button = driver.find_element(By.ID, 'hlkPurchase')
    booking_button.click()
    print("Booking 버튼 클릭 완료")
except NoSuchElementException:
    print("Booking 버튼을 찾을 수 없습니다.")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 새 창으로 제어 전환
try:
    wait.until(EC.number_of_windows_to_be(2))
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except TimeoutException:
    print("새 창이 열리지 않았습니다.")
    driver.quit()
    exit()

# 팝업 처리 after clicking second booking
handle_alert()

# 날짜 선택
try:
    date_button = wait.until(EC.element_to_be_clickable((By.ID, "2024-11-30")))
    date_button.click()
    print("날짜 선택 완료")
except TimeoutException:
    print("날짜 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]')))
    time_button.click()
    print("회차 선택 완료")
except TimeoutException:
    print("회차 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnSeatSelect')))
    seat_button.click()
    print("좌석 선택 버튼 클릭 완료")
except TimeoutException:
    print("좌석 선택 버튼을 찾거나 클릭할 수 없습니다.")
    driver.quit()
    exit()

def switch_to_seat_iframe():
    try:
        seat_iframe = wait.until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "ifrmSeatFrame")))
        print("iframe으로 전환 성공")
    except Exception as e:
        print(f"iframe으로 전환 실패: {e}")
        traceback.print_exc()

# 좌석 선택 및 검증 함수
def select_seat_and_move_next(seat_element):
    try:
        seat_id = seat_element.get_attribute('id')
        seat_title = seat_element.get_attribute('title')
        print(f"좌석 {seat_id} ({seat_title})를 선택 시도합니다.")

        driver.execute_script("arguments[0].click();", seat_element)
        print(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석선택완료 버튼 클릭
        confirm_success = click_confirm()
        if confirm_success:
            print("좌석 선택 완료 및 다음 단계로 이동 성공.")
            return True
        else:
            print("좌석 선택 완료 버튼 클릭 실패. 다시 시도합니다.")
            return False
    except Exception as e:
        print(f"좌석 선택 중 오류 발생: {e}")
        traceback.print_exc()
        return False

# '좌석선택완료' 버튼 클릭 함수
def click_confirm():
    try:
        confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']")))
        confirm_button.click()
        print("좌석선택완료 버튼을 클릭했습니다.")
        return True
    except Exception as e:
        print(f"'좌석선택완료' 버튼 클릭에 실패했습니다: {e}")
        return False
    
def send_telegram_message(message):
    bot_token = '7616270521:AAGXL0x51EOH6iqbuBN9TfFqOWqoSphjKlc'
    chat_id = '7889718434'
    
    send_message_url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    
    params = {
        'chat_id': chat_id,
        'text': message
    }
    
    response = requests.get(send_message_url, params=params)
    
    if response.status_code == 200:
        print("텔레그램 메시지가 성공적으로 전송되었습니다.")
    else:
        print(f"텔레그램 메시지 전송 실패: {response.status_code}, {response.text}")

# 좌석 선택 모니터링 및 시도 함수
# 좌석 선택 모니터링 및 시도 함수
def monitor_and_select_seat():
    attempts = 0
    switch_to_seat_iframe()

    while attempts < MAX_ATTEMPTS:
        try:
            attempts += 1
            print(f"Attempt {attempts} out of {MAX_ATTEMPTS}")

            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade='VIP석']")
            seat_count = len(seats)
            print(f"조건을 만족하는 좌석은 총 {seat_count}개입니다.")

            if seat_count > 0:
                print(f"{seat_count}개의 선택 가능한 좌석을 찾았습니다.")

                for seat in seats:
                    success = select_seat_and_move_next(seat)
                    if success:
                        # 좌석 선택 성공 시 텔레그램 메시지 전송
                        send_telegram_message(f"성공적으로 좌석을 선택했습니다! 좌석 수: {seat_count}")
                        return True  # 좌석 선택 및 다음 단계로 이동 성공
                    else:
                        continue

                print("모든 좌석을 시도했지만 선택하지 못했습니다.")
            else:
                print("선택 가능한 좌석을 찾지 못했습니다.")

        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()

        time.sleep(MONITORING_INTERVAL)

    print("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    send_telegram_message("좌석 선택에 실패했습니다.")
    return False

# '다음단계' 버튼 클릭 함수 (첫 번째 버튼만)
def click_next_step():
    try:
        driver.switch_to.default_content()  # 기본 컨텐츠로 전환

        # '다음단계' 버튼이 화면에 보이고, 상호작용 가능할 때까지 대기
        next_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//img[contains(@src, 'btn_next02.gif')]")))
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//img[contains(@src, 'btn_next02.gif')]")))
        next_button.click()
        print("'다음단계' 버튼을 클릭했습니다.")
        return True
    except Exception as e:
        print(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        return False

if __name__ == "__main__":
    try:
        monitor_and_select_seat()
    except Exception as e:
        print(f"스크립트 실행 중 예기치 않은 오류 발생: {e}")
        traceback.print_exc()
    finally:
        pass


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 128
Current browser version is 130.0.6723.58 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF68BAFB632+29090]
	(No symbol) [0x00007FF68BA6E6E9]
	(No symbol) [0x00007FF68B92B1CA]
	(No symbol) [0x00007FF68B96D6B2]
	(No symbol) [0x00007FF68B96C751]
	(No symbol) [0x00007FF68B96729C]
	(No symbol) [0x00007FF68B961E0A]
	(No symbol) [0x00007FF68B9B236D]
	(No symbol) [0x00007FF68B9B1A20]
	(No symbol) [0x00007FF68B9A6493]
	(No symbol) [0x00007FF68B9709B1]
	(No symbol) [0x00007FF68B971B11]
	GetHandleVerifier [0x00007FF68BE1881D+3294093]
	GetHandleVerifier [0x00007FF68BE64403+3604339]
	GetHandleVerifier [0x00007FF68BE5A2C7+3563063]
	GetHandleVerifier [0x00007FF68BBB6F16+797318]
	(No symbol) [0x00007FF68BA7986F]
	(No symbol) [0x00007FF68BA75454]
	(No symbol) [0x00007FF68BA755E0]
	(No symbol) [0x00007FF68BA64A7F]
	BaseThre